In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections

from tf.app import use
from tf.client.make.build import Make

In [3]:
Mk = Make("missieven", "text", debugState="on")

In [4]:
Mk.loadTf()

In [5]:
A = Mk.A
api = A.api
Fs = api.Fs
F = api.F
L = api.L
T = api.T

In [6]:
T.sectionTypes

['volume', 'page', 'line']

In [18]:
linesx = set()
lines = F.otype.s("line")
print(f"{len(lines)} lines")
         
for ln in F.otype.s("line"):
    fw = L.d(ln, otype="word")[0]
    page = L.u(fw, otype="page")
    if not page:
        linesx.add(ln)
        
len(linesx)

465340 lines


117

# Run the record function

In [43]:
# do this if you have changed mkdata.py or config.yaml

Mk.config()

True

In [29]:
Mk.makeClientSettings()

Node type declared as result focus:

	line

Layers declared as visible in the result ('visible'):

	word/text, word/kind, letter/author, letter/date, letter/place



In [30]:
Mk.makeLinks()

 1h 06m 51s links for types volume, page, line
volume              :     13 links
page                :  10149 links
line                : 465340 links
 1h 07m 07s done


In [44]:
# Mk.config()
Mk.record()

  0.00s preparing ... 
  0.00s start recording
  1 1
  2 2                                                                         
  3 3                                                                         
  4 4                                                                         
  5 5                                                                         
  6 6                                                                         
  7 7                                                                         
  8 8                                                                         
  9 9                                                                         
 10 10                                                                        
 11 11                                                                       
 12 12                                                                       
 13 13                                                                       
  

In [45]:
Mk.dumpCorpus()

  0.00s Dumping data to compact json files
  0.31s Data texts-word-text stored in ~/github/annotation/app-missieven/site/text/corpus/texts-word-text.js
  0.35s Data texts-word-kind stored in ~/github/annotation/app-missieven/site/text/corpus/texts-word-kind.js
  0.36s Data texts-line-location stored in ~/github/annotation/app-missieven/site/text/corpus/texts-line-location.js
  0.36s Data texts-letter-page stored in ~/github/annotation/app-missieven/site/text/corpus/texts-letter-page.js
  0.36s Data texts-letter-author stored in ~/github/annotation/app-missieven/site/text/corpus/texts-letter-author.js
  0.36s Data texts-letter-date stored in ~/github/annotation/app-missieven/site/text/corpus/texts-letter-date.js
  0.36s Data texts-letter-place stored in ~/github/annotation/app-missieven/site/text/corpus/texts-letter-place.js
  0.36s Data texts-volume-number stored in ~/github/annotation/app-missieven/site/text/corpus/texts-volume-number.js
  2.87s Data positions-word-text stored in ~/gi

In [33]:
Mk.dumpConfig()

    26s Config written to file ~/github/annotation/app-missieven/site/text/corpus/config.js


In [34]:
Mk.makeClient()
Mk.adjustDebug()

Copied static files
html file written to /Users/dirk/github/annotation/app-missieven/site/index.html
html file written to /Users/dirk/github/annotation/app-missieven/site/text/index.html
html file (for use with file://) written to /Users/dirk/github/annotation/app-missieven/site/text/index-local.html
Adjusting debug in /Users/dirk/github/annotation/app-missieven/site/text/js/defs.js
Debug set to true
